## Download and Clean Dataset
Let's start by importing the <em>pandas</em> and the <em>Numpy</em> libraries.

In [1]:
import pandas as pd
import numpy as np
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [2]:
concrete_data.shape

(1030, 9)

Let's check the dataset for any missing values.

In [3]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [6]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

# A. Baseline Neural Network
Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


The regression model will have one hidden layer of 10 nodes, and a ReLU activation function. The 'adam' optimizer is used and the 'mean squared error' is used as the loss function.

In [33]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### Train and Test the Network
Let's call the function now to create our model. Importing scikit-learn in order to randomly split the data into a training and test sets

In [34]:
from sklearn.model_selection import train_test_split

#Splitting the data into a training and test sets by holding 30% of the data for testing
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [35]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will train the model for 50 epochs.

In [36]:
# fit the model using the built-in Keras fit function
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 141us/step - loss: 42110.7442
Epoch 2/50
721/721 [==============================] - 0s 39us/step - loss: 9084.1689
Epoch 3/50
721/721 [==============================] - 0s 37us/step - loss: 5846.7455
Epoch 4/50
721/721 [==============================] - 0s 42us/step - loss: 5303.0092
Epoch 5/50
721/721 [==============================] - 0s 47us/step - loss: 4727.7239
Epoch 6/50
721/721 [==============================] - 0s 47us/step - loss: 4196.6743
Epoch 7/50
721/721 [==============================] - 0s 44us/step - loss: 3691.2566
Epoch 8/50
721/721 [==============================] - 0s 43us/step - loss: 3211.1177
Epoch 9/50
721/721 [==============================] - 0s 46us/step - loss: 2782.4639
Epoch 10/50
721/721 [==============================] - 0s 47us/step - loss: 2400.0857
Epoch 11/50
721/721 [==============================] - 0s 41us/step - loss: 2073.8766
Epoch 12/50
721/721 [==============================] - 0s 47u

Next, we need to evaluate the model on the test data.

In [37]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 180us/step


124.70854300588466

Now, we compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [38]:
from sklearn.metrics import mean_squared_error

mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

124.70852710271917 0.0


Creating a list of 50 mean squared errors to report the mean and the standard deviation of the mean squared errors.

In [39]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Herein is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. \nNumber of epochs = " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 93.72215736413851
MSE 2: 119.77520905034828
MSE 3: 115.06454912204187
MSE 4: 123.24235734661806
MSE 5: 122.51923819730197
MSE 6: 108.85042858432412
MSE 7: 133.78014025641878
MSE 8: 98.74063241211728
MSE 9: 130.13444741258343
MSE 10: 110.2504591957265
MSE 11: 106.87584822077581
MSE 12: 101.72823612743983
MSE 13: 117.48222190585336
MSE 14: 115.84360180703567
MSE 15: 109.42565019231013
MSE 16: 106.54830542505752
MSE 17: 109.85340202356234
MSE 18: 95.46041724442664
MSE 19: 95.13645241252813
MSE 20: 113.96541988039479
MSE 21: 107.87222625831184
MSE 22: 104.10498859195647
MSE 23: 108.07317715246701
MSE 24: 103.69747660615297
MSE 25: 129.71703247267837
MSE 26: 117.16039086005449
MSE 27: 120.7952090760265
MSE 28: 112.21814345078947
MSE 29: 109.34347852688391
MSE 30: 118.15906255530693
MSE 31: 131.64821234644424
MSE 32: 118.67370533866018
MSE 33: 102.13411823754171
MSE 34: 116.58001689231897
MSE 35: 114.86474525426969
MSE 36: 140.55356052238193
MSE 37: 123.19497423881852
MSE 38: 125.3785

# B. Normalize the data

In [15]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()
n_cols = predictors_norm.shape[1] # number of predictors

### Train and Test the Network
Let's call the function now to create our model. Importing scikit-learn in order to randomly split the data into a training and test sets

In [16]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [17]:
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 43us/step - loss: 1641.4474
Epoch 2/50
721/721 [==============================] - 0s 39us/step - loss: 1600.4436
Epoch 3/50
721/721 [==============================] - 0s 46us/step - loss: 1566.2529
Epoch 4/50
721/721 [==============================] - 0s 47us/step - loss: 1531.7547
Epoch 5/50
721/721 [==============================] - 0s 47us/step - loss: 1495.8367
Epoch 6/50
721/721 [==============================] - 0s 54us/step - loss: 1458.5488
Epoch 7/50
721/721 [==============================] - 0s 45us/step - loss: 1418.4269
Epoch 8/50
721/721 [==============================] - 0s 50us/step - loss: 1378.1879
Epoch 9/50
721/721 [==============================] - 0s 42us/step - loss: 1335.7395
Epoch 10/50
721/721 [==============================] - 0s 41us/step - loss: 1292.5405
Epoch 11/50
721/721 [==============================] - 0s 44us/step - loss: 1248.2411
Epoch 12/50
721/721 [==============================] - 0s 44us/

Next, we need to evaluate the model on the test data.

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 29us/step


296.69723016930243

Now, we compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [19]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

296.69723382110055 0.0


Creating a list of 50 mean squared errors to report the mean and the standard deviation of the mean squared errors.

In [20]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Herein is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. \nNumber of epochs = " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 147.06541447809212
MSE 2: 127.79589384504892
MSE 3: 87.87909784903418
MSE 4: 82.39735889126182
MSE 5: 77.38637690868192
MSE 6: 73.48410173681562
MSE 7: 69.73554323560597
MSE 8: 48.51257690874118
MSE 9: 48.83147613130341
MSE 10: 48.4530081640941
MSE 11: 42.90127640017414
MSE 12: 40.83138068979998
MSE 13: 48.319239662688915
MSE 14: 48.255879510182005
MSE 15: 37.81736849195363
MSE 16: 35.82292818483025
MSE 17: 38.52632178766442
MSE 18: 37.383743360204605
MSE 19: 36.53798945899148
MSE 20: 37.20282977452942
MSE 21: 33.390151138058755
MSE 22: 37.54053449167789
MSE 23: 29.980434343652817
MSE 24: 33.26540475987308
MSE 25: 36.24604085656817
MSE 26: 36.27933393558638
MSE 27: 31.328015299676693
MSE 28: 30.084678223989542
MSE 29: 37.685283870758745
MSE 30: 33.64626071522537
MSE 31: 32.12910296616045
MSE 32: 30.894642450277086
MSE 33: 31.117301150818857
MSE 34: 33.24824667131245
MSE 35: 32.55411062888729
MSE 36: 38.54751278281598
MSE 37: 29.90651423259846
MSE 38: 34.87061969053398
MSE 39: 33

### We see that the mean in Step B is lesser than the mean of Step A.

# C. Increase the number of epochs
Repeating Part B but use 100 epochs this time for training.

### Train and Test the Network
Let's call the function now to create our model. Importing scikit-learn in order to randomly split the data into a training and test sets

In [21]:
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
721/721 [==============================] - 0s 43us/step - loss: 28.3989
Epoch 2/100
721/721 [==============================] - 0s 41us/step - loss: 28.4760
Epoch 3/100
721/721 [==============================] - 0s 38us/step - loss: 28.4952
Epoch 4/100
721/721 [==============================] - 0s 44us/step - loss: 28.4173
Epoch 5/100
721/721 [==============================] - 0s 42us/step - loss: 28.4045
Epoch 6/100
721/721 [==============================] - 0s 42us/step - loss: 28.4295
Epoch 7/100
721/721 [==============================] - 0s 43us/step - loss: 28.3927
Epoch 8/100
721/721 [==============================] - 0s 40us/step - loss: 28.3968
Epoch 9/100
721/721 [==============================] - 0s 42us/step - loss: 28.4307
Epoch 10/100
721/721 [==============================] - 0s 39us/step - loss: 28.3594
Epoch 11/100
721/721 [==============================] - 0s 43us/step - loss: 28.4380
Epoch 12/100
721/721 [==============================] - 0s 37us/step - los

721/721 [==============================] - 0s 32us/step - loss: 28.0607
Epoch 98/100
721/721 [==============================] - 0s 38us/step - loss: 28.1058
Epoch 99/100
721/721 [==============================] - 0s 33us/step - loss: 28.0919
Epoch 100/100
721/721 [==============================] - 0s 32us/step - loss: 28.0393


Next, we need to evaluate the model on the test data.

In [22]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 26us/step


33.55373658485783

Now, we compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [23]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

33.55373488654564 0.0


Creating a list of 50 mean squared errors to report the mean and the standard deviation of the mean squared errors.

In [24]:
total_mean_squared_errors = 50
epochs = 100
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Herein is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. \nNumber of epochs = " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 32.1973007473745
MSE 2: 35.455000275547064
MSE 3: 26.081790553713308
MSE 4: 30.242432659112133
MSE 5: 30.25621379617734
MSE 6: 33.10248649621859
MSE 7: 34.60612361870923
MSE 8: 29.605429183318005
MSE 9: 30.808645810124172
MSE 10: 32.96847831701383
MSE 11: 31.599708587992154
MSE 12: 27.985727865719102
MSE 13: 33.94846687810706
MSE 14: 36.70049822909161
MSE 15: 28.06329162375441
MSE 16: 26.94378023240173
MSE 17: 29.268135527576828
MSE 18: 31.124168587348223
MSE 19: 30.921901505356082
MSE 20: 28.505434968324927
MSE 21: 29.993937372003945
MSE 22: 29.24036350065065
MSE 23: 27.048372052634033
MSE 24: 28.400051771244186
MSE 25: 31.506168816081914
MSE 26: 31.815420860611503
MSE 27: 28.806089697532283
MSE 28: 24.958782368879103
MSE 29: 34.161597180906625
MSE 30: 31.577035903930664
MSE 31: 28.389841252546095
MSE 32: 25.73035934596386
MSE 33: 26.388080868520397
MSE 34: 30.945640736799024
MSE 35: 30.10694280334275
MSE 36: 34.19090615429924
MSE 37: 27.660108637269648
MSE 38: 30.9777937953911

### We see that the mean in Step C is lesser than the mean of Step B.

# D. Increase the number of hidden layers
Repeating part B but using a neural network with 3 hidden layers, each of 10 nodes and ReLU activation function.

In [25]:
def regression_model2():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### Train and Test the Network
Let's call the function now to create our model. Importing scikit-learn in order to randomly split the data into a training and test sets

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [27]:
# build the model
model2 = regression_model2()

In [28]:
model2.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 158us/step - loss: 1535.2431
Epoch 2/50
721/721 [==============================] - 0s 44us/step - loss: 1482.7345
Epoch 3/50
721/721 [==============================] - 0s 47us/step - loss: 1414.7360
Epoch 4/50
721/721 [==============================] - 0s 43us/step - loss: 1318.4525
Epoch 5/50
721/721 [==============================] - 0s 48us/step - loss: 1181.8122
Epoch 6/50
721/721 [==============================] - 0s 48us/step - loss: 1007.8773
Epoch 7/50
721/721 [==============================] - 0s 43us/step - loss: 811.2789
Epoch 8/50
721/721 [==============================] - 0s 47us/step - loss: 629.7163
Epoch 9/50
721/721 [==============================] - 0s 44us/step - loss: 488.0801
Epoch 10/50
721/721 [==============================] - 0s 43us/step - loss: 388.4209
Epoch 11/50
721/721 [==============================] - 0s 41us/step - loss: 326.4605
Epoch 12/50
721/721 [==============================] - 0s 39us/step

Next, we need to evaluate the model on the test data.

In [29]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 35us/step


25.637830524382853

Now, we compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [30]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

25.637831173122994 0.0


Creating a list of 50 mean squared errors to report the mean and the standard deviation of the mean squared errors.

In [31]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Herein is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. \nNumber of epochs = " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 28.548978527772775
MSE 2: 33.430840254601534
MSE 3: 24.36949998429678
MSE 4: 27.3547257975853
MSE 5: 27.244276194896514
MSE 6: 31.78864911767657
MSE 7: 32.72632874177112
MSE 8: 27.609818190046884
MSE 9: 29.72259511608136
MSE 10: 28.612551179904383
MSE 11: 29.379796722560254
MSE 12: 27.224457503136694
MSE 13: 31.64892359613215
MSE 14: 33.19884695898754
MSE 15: 27.10233917977046
MSE 16: 24.844036670178657
MSE 17: 27.05734380703528
MSE 18: 28.74505141792174
MSE 19: 29.042218118809572
MSE 20: 26.60146336725229
MSE 21: 28.034851148290542
MSE 22: 27.89334674329048
MSE 23: 25.45762499713589
MSE 24: 27.14217556950344
MSE 25: 30.552818921777423
MSE 26: 30.090431133134466
MSE 27: 26.959693544505097
MSE 28: 24.95158086239713
MSE 29: 33.04095168869858
MSE 30: 30.228568302389103
MSE 31: 26.419417563379774
MSE 32: 24.086218046910556
MSE 33: 24.50766471443053
MSE 34: 29.96185242242412
MSE 35: 28.197562060309846
MSE 36: 33.35152326046842
MSE 37: 26.75168337713939
MSE 38: 29.82188845143735
MSE 3

### We see that the mean in Step D is lesser than the mean of Step B.